In [1]:
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd

In [7]:
def probAttr(data:pd.DataFrame, attr:str, val:str):
    Total = data.shape[0]
    cnt = len(data[data[attr] == val])
    # cnt for each yes and no
    # counts the occurances
    # the function returns 9, 9/14 for 'Yes' 
    # the function returns 5, 5/14 for 'No'
    return cnt, cnt / Total

In [8]:
#data -> whole playtennis
#Attr -> {'Outlook':     {'Sunny', 'Rain', 'Overcast'}, 
#         'Temperature': {'Mild', 'Hot', 'Cool'}, 
#         'Humidity':    {'Normal', 'High'}, 
#         'Wind':        {'Strong', 'Weak'}
#}
#targetVals -> {'No', 'Yes'}
#concept -> 'PlayTennis' (target column name)
def train(data:pd.DataFrame, Attr:dict, targetVals:set, concept:str):
    conceptProbs = {}
    countConcept = {}
    for cVal in targetVals:
        countConcept[cVal], conceptProbs[cVal] = probAttr(data, concept, cVal)
    AttrConcept = {}
    probability_list = {}
    for att in Attr:
        probability_list[att] = {}
        AttrConcept[att] = {}
        for val in Attr[att]:
            AttrConcept[att][val] = {}
            _, probability_list[att][val] = probAttr(data, att, val)
            for cVal in targetVals:
                dataTemp = data[data[att] == val]
                AttrConcept[att][val][cVal] = len(dataTemp[dataTemp[concept] == cVal]) / countConcept[cVal]
    print(f"P(A) : {conceptProbs}\n")
    print(f"P(X/A) : {AttrConcept}\n")
    print(f"P(X) : {probability_list}\n")
    return conceptProbs, AttrConcept, probability_list


In [9]:
def test(examples, Attr, concept_list, conceptProbs, AttrConcept,probability_list):
    misclassification_count = 0
    Total = len(examples)
    for ex in examples:
        px = {}
        for a in Attr:
            for x in ex:
                for c in concept_list:
                    if x in AttrConcept[a]:
                        if c not in px:
                            px[c] = conceptProbs[c] * AttrConcept[a][x][c] / probability_list[a][x]
                        else:
                            px[c] = px[c] * AttrConcept[a][x][c] / probability_list[a][x]
        print(px)
        classification = max(px, key=px.get)
        print(f"Classification : {classification} Expected : {ex[-1]}")
        if (classification != ex[-1]):
            misclassification_count += 1
            
    misclassification_rate = misclassification_count * 100 / Total
    accuracy = 100 - misclassification_rate
    print(f"Misclassification Count={misclassification_count}")
    print(f"Misclassification Rate={misclassification_rate}%")
    print(f"Accuracy={accuracy}%")


In [12]:
df = pd.read_csv('playtennnis.csv')
target = str(list(df)[-1])
print('target ->', target)
target_list = set(df[target]) # Yes and No only. 
Attr = {}
for a in df.columns[:-1]:
    Attr[a] = set(df[a])
    # print(f"{a}: {Attr[a]}")
print('Attr ->', Attr)
conceptProbs, AttrConcept, probability_list = train(df, Attr, target_list, target)
examples = pd.read_csv('playtennnis.csv')
test(examples.values, Attr, target_list, conceptProbs, AttrConcept, probability_list)
# df

target -> PlayTennis
Attr -> {'Outlook': {'Sunny', 'Rain', 'Overcast'}, 'Temperature': {'Mild', 'Hot', 'Cool'}, 'Humidity': {'Normal', 'High'}, 'Wind': {'Strong', 'Weak'}}
P(X) : {'Outlook': {'Sunny': 0.35714285714285715, 'Rain': 0.35714285714285715, 'Overcast': 0.2857142857142857}, 'Temperature': {'Mild': 0.42857142857142855, 'Hot': 0.2857142857142857, 'Cool': 0.2857142857142857}, 'Humidity': {'Normal': 0.5, 'High': 0.5}, 'Wind': {'Strong': 0.42857142857142855, 'Weak': 0.5714285714285714}}

{'Yes': 0.2419753086419753, 'No': 0.9408000000000002}
Classification : No Expected : No
{'Yes': 0.16131687242798354, 'No': 1.8816000000000002}
Classification : No Expected : No
{'Yes': 0.6049382716049383, 'No': 0.0}
Classification : Yes Expected : Yes
{'Yes': 0.4839506172839506, 'No': 0.4181333333333335}
Classification : Yes Expected : Yes
{'Yes': 1.0888888888888888, 'No': 0.07840000000000004}
Classification : Yes Expected : Yes
{'Yes': 0.7259259259259259, 'No': 0.15680000000000005}
Classification 